# Yolo-v2 DW2TF validation

In [1]:
import numpy as np
import tensorflow as tf

in_data = np.random.randn(64, 608, 608, 3) * 120.56

In [2]:
tf.reset_default_graph()

tf.train.import_meta_graph('/data/darkflow/yolov2.ckpt.meta')
ckpt_path = '/data/darkflow/yolov2.ckpt'

g = tf.get_default_graph()
in_t = g.get_tensor_by_name('input:0')
out_t = g.get_tensor_by_name('output:0')

with tf.Session() as sess:
    var_list = {} # dictionary mapping variable name to tensor, used to create saver object to restore
    reader = tf.train.NewCheckpointReader(ckpt_path)
    for key in reader.get_variable_to_shape_map():
        # Look for all variables in ckpt that are used by the graph
        try:
            tensor = g.get_tensor_by_name(key + ":0")
        except KeyError:
            # This tensor doesn't exist in the graph (for example it's
            # 'global_step' or a similar housekeeping element) so skip it.
            continue
        var_list[key] = tensor
    saver = tf.train.Saver(var_list=var_list)
    saver.restore(sess, ckpt_path)

    out_data_darkflow = sess.run(out_t, feed_dict={in_t: in_data})

INFO:tensorflow:Restoring parameters from /data/darkflow/yolov2.ckpt


In [3]:
tf.reset_default_graph()

tf.train.import_meta_graph('/data/DW2TF/data/yolov2.ckpt.meta')
ckpt_path = '/data/DW2TF/data/yolov2.ckpt'

g = tf.get_default_graph()
in_t = g.get_tensor_by_name('yolov2/net1:0')
out_t = g.get_tensor_by_name('yolov2/convolutional23/BiasAdd:0')

with tf.Session() as sess:
    var_list = {} # dictionary mapping variable name to tensor, used to create saver object to restore
    reader = tf.train.NewCheckpointReader(ckpt_path)
    for key in reader.get_variable_to_shape_map():
        # Look for all variables in ckpt that are used by the graph
        try:
            tensor = g.get_tensor_by_name(key + ":0")
        except KeyError:
            # This tensor doesn't exist in the graph (for example it's
            # 'global_step' or a similar housekeeping element) so skip it.
            continue
        var_list[key] = tensor
    saver = tf.train.Saver(var_list=var_list)
    saver.restore(sess, ckpt_path)
    
    out_data_dw2tf = sess.run(out_t, feed_dict={in_t: in_data})

INFO:tensorflow:Restoring parameters from /data/DW2TF/data/yolov2.ckpt


In [4]:
print(out_data_darkflow.shape)
print(out_data_dw2tf.shape)
print(np.mean(out_data_darkflow - out_data_dw2tf)**2)

(64, 19, 19, 425)
(64, 19, 19, 425)
2.2677931848742855e-15


In [5]:
out_data_darkflow

array([[[[-4.90138435e+00,  3.36968064e+00, -5.34612179e+00, ...,
          -6.10880232e+00, -4.40329885e+00, -7.12015438e+00],
         [-2.58250284e+00,  1.16267805e+01, -6.64636230e+00, ...,
          -7.25774193e+00, -2.49555635e+00, -9.57235432e+00],
         [-3.69526029e+00,  1.03629684e+01, -6.54407692e+00, ...,
          -1.13449469e+01, -2.31868124e+00, -1.22897596e+01],
         ...,
         [-3.98463702e+00,  1.12876530e+01, -8.89100170e+00, ...,
          -8.91366673e+00, -2.58457518e+00, -1.27230225e+01],
         [-3.19200253e+00,  1.19259033e+01, -8.47108746e+00, ...,
          -1.09786072e+01, -2.11832619e+00, -1.15288763e+01],
         [ 2.80406475e-01,  9.17492771e+00, -5.41952801e+00, ...,
          -6.39503717e+00, -1.53763008e+00, -6.66333580e+00]],

        [[-4.34767771e+00,  2.33599573e-01, -7.24404716e+00, ...,
          -4.55735731e+00, -5.46119118e+00, -1.14438715e+01],
         [-7.69979835e-01, -6.91370487e-01, -7.67613363e+00, ...,
          -9.85900021e

In [6]:
out_data_dw2tf

array([[[[-4.90133286e+00,  3.36972833e+00, -5.34607649e+00, ...,
          -6.10879803e+00, -4.40329361e+00, -7.12012243e+00],
         [-2.58249974e+00,  1.16268053e+01, -6.64634705e+00, ...,
          -7.25777864e+00, -2.49555731e+00, -9.57233334e+00],
         [-3.69525695e+00,  1.03629971e+01, -6.54406977e+00, ...,
          -1.13449516e+01, -2.31868529e+00, -1.22897615e+01],
         ...,
         [-3.98463035e+00,  1.12876844e+01, -8.89099121e+00, ...,
          -8.91370869e+00, -2.58458042e+00, -1.27230520e+01],
         [-3.19200563e+00,  1.19258957e+01, -8.47109032e+00, ...,
          -1.09786263e+01, -2.11832619e+00, -1.15288820e+01],
         [ 2.80440748e-01,  9.17492771e+00, -5.41951323e+00, ...,
          -6.39508486e+00, -1.53762329e+00, -6.66333485e+00]],

        [[-4.34766150e+00,  2.33608961e-01, -7.24404764e+00, ...,
          -4.55737305e+00, -5.46118736e+00, -1.14438610e+01],
         [-7.69983232e-01, -6.91357791e-01, -7.67612600e+00, ...,
          -9.85905743e

# Reorg layer test

In [7]:
# Reorg layer test

batch_size, height, width, channels = 1, 2, 4, 1
stride = 2

_height, _width, _channel = height // stride, width // stride, channels * stride * stride

input = np.array([[[[0], [2], [1], [3]], [[4], [6], [5], [7]]]])

net = input
net = tf.reshape(net, [batch_size, _height, stride, _width, stride, channels])
net = tf.transpose(net, [0, 1, 3, 2, 4, 5])  # batch_size, _height, _width, stride, stride, channels
net = tf.reshape(net, [batch_size, _height, _width, stride * stride * channels], name='reorg')

output = net.eval(session=tf.Session())

print(input.shape)
print(output.shape)

(1, 2, 4, 1)
(1, 1, 2, 4)


In [8]:
input

array([[[[0],
         [2],
         [1],
         [3]],

        [[4],
         [6],
         [5],
         [7]]]])

In [9]:
output

array([[[[0, 2, 4, 6],
         [1, 3, 5, 7]]]])

In [10]:
assert input[0,0,0,0] == output[0,0,0,0]
assert input[0,0,1,0] == output[0,0,0,1]
assert input[0,0,2,0] == output[0,0,1,0]
assert input[0,0,3,0] == output[0,0,1,1]
assert input[0,1,0,0] == output[0,0,0,2]
assert input[0,1,1,0] == output[0,0,0,3]
assert input[0,1,2,0] == output[0,0,1,2]
assert input[0,1,3,0] == output[0,0,1,3]

# Read darknet .weights

In [11]:
weight_array = np.fromfile('/data/darknet/yolov2.weights', dtype='f')

In [12]:
weight_array.shape


(50983565,)